# Text generation using Hidden Markov Model

For this project I will generate new text and perform text prediction using the Hidden Markov Model based of ABC news headline.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
headline_df = pd.read_csv("../data/abcnews-date-text.csv") 

## Data cleaning 

As the data set is too large with over a million rows and it takes too long to run and train the model. I decided to sample 99999 rows out of the million+ rows.

In [3]:
headline_df = headline_df[['headline_text']]
headline_df = headline_df.sample(99999)
headline_df

headline_text
390544     minister calls for upfront parking fine tactics
720474           sugar industry cant afford looming strike
97949     convent to play host to arts and cultural centre
595117             food shortages cruel kims birthday bash
9249                     youths await sentencing over rape
...                                                    ...
1018604  twitter australia md on the future and living ...
83163         govt urged to consider light rail system for
875572   looming broiler code review promises more plan...
60366    recommendations made to improve safety for police
1122600  china lists of us goods it might hit with 25 p...

[99999 rows x 1 columns]

## Formulate ideas on how ML can be used to learn word correlations and distributions

Idea 1: A possible Machine Learning algorithm that comes to mind that may be used to learn word correlations and distributions would be the K-means algorithm. By using K-means, we can distribute text and words that are correlated to each other into clusters. Therefore, basing the word prediction by determining which cluster a given word belongs to. (Just an idea)

Idea 2: Another idea would be to use the Markov Model. As words and text are sequential data, representing its correlation and distribution using a Markov Model is intuitive. However, often times the states we want to understand are hidden such as part-of-speech tags when modeling text data. Therefore, by including hidden states (hence using the Hidden Markov Model) it allows us to use observed and hidden states as a factor when determining the probability of the next generated word. This is what we will be building for this project.

## Building Hidden Markov Model

### 1. Collecting all the different words from the dataset

In [4]:
words = []
headlines = headline_df['headline_text']

for headline in headlines:
    headline = headline.split()
    for word in headline:
        words.append(word)
        
distinct_words = list(set(words))
distinct_words.append(None)  # Null State
distinct_words_count = len(distinct_words)
word_dict = {word: i for i, word in enumerate(distinct_words)}

### 2. Initializing and defining the transition matrix

In [5]:
matrix_1, matrix_2 = np.zeros((distinct_words_count, distinct_words_count)), np.zeros((distinct_words_count, distinct_words_count))

for headline in headlines:
    data = headline.split()
    for i in range(len(data)):
        if i < len(data) - 1:
            matrix_1[word_dict[data[i]]][word_dict[data[i + 1]]] += 1
        else:
            matrix_1[word_dict[data[i]]][distinct_words_count - 1] += 1

        if i < len(data) - 2:
            matrix_2[word_dict[data[i]]][word_dict[data[i + 2]]] += 1
        else:
            matrix_2[word_dict[data[i]]][distinct_words_count - 1] += 1

matrix_1[distinct_words_count - 1][distinct_words_count - 1], matrix_2[distinct_words_count - 1][distinct_words_count - 1] = 1, 1

for i in range(len(matrix_1)):
    matrix_1[i], matrix_2[i]= matrix_1[i] / matrix_1[i].sum(), matrix_2[i] / matrix_2[i].sum()

### 3. Implementing the hidden states

Through the research papers I read, most authors uses part-of-speech tags as their hidden state when building a Hidden Markov Model for text generation. However, I am unsure how to implement part-of-speech tags as the hidden state as it is not being labeled in the dataset I chose.

Therefore, I will approach it differently. For the purpose of this project I will have my hidden states as words that either start with a vowels or non-vowels. There are approximated 170,000 words that are currently being used in the english language and I will assume that 10% of them starts with a vowel. Therefore, the probability of a word being followed by another word that starts with a vowel is far lower.

In [ ]:
hidden_dict = {'vowel': 0, 'non-vowel': 1}
hidden_states = ['vowel','non-vowel']
hidden_matrix = [[.8, .2],[.9, .1]]

def emission(probability, state):
    for i in range(len(probability)-1):
        if state is 'vowel':
            if distinct_words[i][0] == 'a' or distinct_words[i][0] == 'e' or distinct_words[i][0] == 'i' or distinct_words[i][0] == 'o' or distinct_words[i][0] == 'u':
                probability[i] *= 2
            else:
                probability[i] /= 2
        else:
            if distinct_words[i][0] == 'a' or distinct_words[i][0] == 'e' or distinct_words[i][0] == 'i' or distinct_words[i][0] == 'o' or distinct_words[i][0] == 'u':
                probability[i] /= 2
            else:
                probability[i] *= 2
    probability[i] /= probability.sum()


def nextHiddenState(hiddenState):
    nextHiddenState = np.random.choice(hiddenStates, size=1, p=hiddenStateTransitionMatrix[hiddenStateDict[hiddenState]])
    return nextHiddenState[0]

### 4. Sampling matrix

In [ ]:
def HMM(word_choice = None, length = 7, sentence = [], hidden_State = 'Short'):
    if word_choice is None:
        word_choice = np.random.choice(new_words, size=1)[0]
    if sentence == []:
        sentence.append(word_choice)   
    next_word = np.random.choice(unique_words, size = 1,p = First_Trans_Mat[words[sentence[-1]]])[0]
    if length > 1:
        while(next_word is None):
            next_word = np.random.choice(unique_words, size = 1,p = First_Trans_Mat[words[sentence[-1]]])[0]

    while next_word is not None:
        sentence.append(next_word)
        next_Probs = First_Trans_Mat[words[sentence[-1]]] * (Second_Trans_Mat[words[sentence[-2]]]) + First_Trans_Mat[words[sentence[-1]]]/4
        next_prob = emission(next_Probs, hidden_State)
        next_Probs[-1] += 0.00001
        next_Probs = next_Probs / next_Probs.sum()
        
        if len(sentence) < length - 1:
            if next_Probs.sum() > next_Probs[-1]:
                next_Probs[-1] = next_Probs[-1] / 10
            next_Probs = next_Probs / next_Probs.sum()
            
        if len(sentence) > length + 1:
            next_Probs[-1] = next_Probs[-1] * 2
            next_Probs = next_Probs / next_Probs.sum()
        next_word = np.random.choice(unique_words, size = 1,p = next_Probs)[0]
        
        hidden_State = nextState(hidden_State)
        
    return sentence

def generateText(init_word, text_arr):
    if init_word == "":
        init_word = np.random.choice(distinct_word)
    text_arr.append(init-word)
    following_word = np.random.choice(distinct_words, p = matrix_1[word_dict[sentence[-1]]])
    
    for i in range(8):
        sentence.append(next_word)
        next_Probs = First_Trans_Mat[words[sentence[-1]]] * (Second_Trans_Mat[words[sentence[-2]]]) + First_Trans_Mat[words[sentence[-1]]]/4
        next_prob = emission(next_Probs, hidden_State)
        next_Probs[-1] += 0.00001
        next_Probs = next_Probs / next_Probs.sum()
        
        if len(sentence) < length - 1:
            if next_Probs.sum() > next_Probs[-1]:
                next_Probs[-1] = next_Probs[-1] / 10
            next_Probs = next_Probs / next_Probs.sum()
            
        if len(sentence) > length + 1:
            next_Probs[-1] = next_Probs[-1] * 2
            next_Probs = next_Probs / next_Probs.sum()
        next_word = np.random.choice(unique_words, size = 1,p = next_Probs)[0]
        
        hidden_State = nextState(hidden_State)
        